In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
import chainer
from chainer import datasets
from chainer import functions as F
from chainer import links as L
from chainer import Variable
from chainer.backends import cuda
from sklearn.model_selection import train_test_split
xp = cuda.cupy

In [3]:
%%time
train_df = pd.read_csv("../input/train.csv")
t = []
for s in train_df['comment_text']:
    t.append(s.split())
words = {}
for j in range(len(t)):
    for word in t[j]:
        if word not in words:
            words[word] = len(words)
t_vec = []
for j in range(len(t)):
    t_ids = []
    for word in t[j]:
        t_ids.append(words[word])
    t_ids = xp.array(t_ids, dtype=xp.int32)
    t_vec.append(t_ids)
del t, word

CPU times: user 4min 10s, sys: 7.98 s, total: 4min 18s
Wall time: 4min 18s


In [4]:
%%time
max_len = max(list(map(len, t_vec)))
for i in range(len(t_vec)):
    add = xp.zeros((max_len-len(t_vec[i])), dtype=xp.int32)
    t_vec[i] = xp.hstack((t_vec[i], add))

CPU times: user 3min 9s, sys: 1.54 s, total: 3min 11s
Wall time: 3min 11s


In [5]:
X = t_vec
y= np.where(train_df['target']>0.5, 1, 0)
y = xp.array(y, dtype=xp.int32)
del train_df, t_vec, t_ids, add

In [6]:
class jigsaw_dataset(chainer.dataset.DatasetMixin):
    def __init__(self, X, y, train=True, min_count=10, train_size=0.6):
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, train_size=train_size, random_state=1)
        self.n_train = len(self.y_train)
        self.n_test = len(self.y_test)
        self.train = train
        self.train_size=train_size
    
    def __len__(self):
        if self.train:
            return self.n_train
        else:
            return self.n_test
    
    def get_example(self, i):
        if self.train:
            train = datasets.tuple_dataset.TupleDataset(self.X_train, self.y_train)
            return train[i]
        else:
            test = datasets.tuple_dataset.TupleDataset(self.X_test, self.y_test)
            return test[i]


In [7]:
class RNN(chainer.Chain):
    def __init__(self, n_vocab=1670966, n_units=100):
        super(RNN, self).__init__(
            embed = L.EmbedID(n_vocab, n_units),
            l1 = L.LSTM(None, n_units),
            l2 = L.LSTM(None, n_units),
            l3 = L.Linear(None, 2)
        )
    
    def reset_state(self):
        self.l1.reset_state()
        self.l2.reset_state()

    def forward(self, x):
        h0 = self.embed(x)
        h1 = self.l1(F.dropout(h0))
        h2 = self.l2(F.dropout(h1))
        y = F.softmax(self.l3(h2))
        return y


model = L.Classifier(RNN())

gpu_id = 0
if gpu_id >= 0:
    model.to_gpu(gpu_id)
    
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer.GradientClipping(5))

In [8]:
BATCH_SIZE = 10
MAX_EPOCH = 10

In [9]:
%%time
train = jigsaw_dataset(X, y)
test = jigsaw_dataset(X, y, train=False)
del X, y

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


CPU times: user 1.75 s, sys: 40 ms, total: 1.79 s
Wall time: 1.87 s


In [11]:
train_iter = chainer.iterators.SerialIterator(train, BATCH_SIZE)
test_iter = chainer.iterators.SerialIterator(test, BATCH_SIZE, repeat=False, shuffle=False)
del train, test

In [17]:
updater = chainer.training.updaters.StandardUpdater(train_iter, optimizer, device=gpu_id)
trainer = chainer.training.Trainer(updater, (MAX_EPOCH, 'epoch'))
trainer.extend(chainer.training.extensions.Evaluator(test_iter, model, device=gpu_id))
trainer.extend(chainer.training.extensions.LogReport(trigger=(1, "epoch")))
trainer.extend(chainer.training.extensions.PrintReport(["epoch", "main/loss", "validation/main/loss", "main/accuracy", "validation/main/accuracy", "elapsed_time"]))

In [18]:
%%time
trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time


KeyboardInterrupt: 

In [ ]:
sum_perp = 0
c = 0
while train_iter.epoch < MAX_EPOCH:
    loss = 0
    '''
    for i in range(35):
        train_batch = train_iter.next()
#         sentence_train, target_train = chainer.dataset.concat_examples(train_batch)
        sentence_train, target_train = chainer.dataset.concat_examples(train_batch, gpu_id)
#         sentence_train, target_train = chainer.dataset.concat_examples(train_iter.next(), gpu_id)


#         prediction_train = model(sentence_train)
#         loss += optimizer.target(prediction_train, target_train)
        loss += optimizer.target(sentence_train, target_train)

        print(0)
#         loss = F.softmax_cross_entropy(model(sentence_train), target_train)
    
    sum_perp += loss.array
#     model.cleargrads()
    optimizer.target.cleargrads()
    print(1)
    loss.backward()
    loss.unchain_backward()    
    optimizer.update()
    '''
    train_batch = train_iter.next()
    sentence_train, target_train = chainer.dataset.concat_examples(train_batch, gpu_id)
    loss += optimizer.target(sentence_train, target_train)
    sum_perp += loss.array
    optimizer.target.cleargrads()
    loss.backward()
    loss.unchain_backward()    
    optimizer.update()
    print(c)
    c = c + 1

    if train_iter.is_new_epoch:
        print('epoch:{:02d} train_loss:{:.04f}'.format(train_iter.epoch, float(loss.array)), end='')
        
        test_losses = []
        test_accuracies = []
        while True:
            loss_test = 0
            for i in ragen(35):
                test_batch = test_iter.next()
#                 sentence_test, target_test = chainer.dataset.concat_examples(test_batch)
                sentence_test, target_test = chainer.dataset.concat_examples(test_batch, gpu_id)
#                 sentence_test, target_test = chainer.dataset.concat_examples(test_iter.next(), gpu_id)

                prediction_test = model(sentence_test)
                loss_test += optimizer.target(prediction_test, target_test)
#                 loss_test = F.mean_squared_error(model(sentence_test), target_test)
            
            test_losses.append(loss_test.array)
#             test_losses.append(cuda.to_cpu(loss_test.array))
            
            accuracy = F.accuracy(prediction_test, target_test)
            accuracy.cuda.to_cpu()#
            test_accuracies.append(accuracy.array)
            
            if test_iter.is_new_epoch:
                test_iter.epoch = 0
                test_iter.current_position = 0
                test_iter.is_new_epoch = False
                test_iter._pushed_position = None
                break;
        
        print('val_loss:{:.04f} val_accuracy:{:.04f}'.format(np.mean(test_losses), np.mean(test_accuracies)))